### Making HTTP Requests

The way we interact with web sites and web servers (including RESTful JSON APIs) is through HTTP **requests** and **responses.**

We can use the `requests` library to make http requests. This is somewhat the same as visiting a url in your browser, except that we can interact with the responses programatically in python.

In [1]:
import requests

We will use the `get` function from `requests` and pass it a url:

In [2]:
response = requests.get('https://example.com')
response

<Response [200]>

We get back a **python object that represents an HTTP response.**

The response object has several interesting properties:

- `.ok`: a boolean that indicates that the response was successful (the server sent back a 200 response code)
- `.status_code`: a number indicating the HTTP response status code
- `.text`: the raw response text

In [3]:
response.ok

True

In [4]:
response.status_code

200

In [5]:
response.text

'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    <

In this case, we see a string that contains HTML. HTML is what makes up web pages that are intended for humans to read. If you go to http://example.com, you'll see what the HTML in the above response looks like when rendered. Some other endpoints on the internet return JSON, which is usually intended to be worked with programatically.

### Example JSON API
For an example of a JSON api, we'll interact with the a [**quote generator**](https://aphorisms.glitch.me).

In [8]:
url = 'https://swapi.dev/api/people/5'
response = requests.get(url)
print(response.text)

{"name":"Leia Organa","height":"150","mass":"49","hair_color":"brown","skin_color":"light","eye_color":"brown","birth_year":"19BBY","gender":"female","homeworld":"http://swapi.dev/api/planets/2/","films":["http://swapi.dev/api/films/1/","http://swapi.dev/api/films/2/","http://swapi.dev/api/films/3/","http://swapi.dev/api/films/6/"],"species":[],"vehicles":["http://swapi.dev/api/vehicles/30/"],"starships":[],"created":"2014-12-10T15:20:09.791000Z","edited":"2014-12-20T21:17:50.315000Z","url":"http://swapi.dev/api/people/5/"}


Here we see that the repsonse we got back contains a JSON object (we could also verify this by visiting the URL in a web browser).

Since the response is JSON, we can use the `.json` method on the response object to get a data structure we can work with:

In [9]:
data = response.json()
print(type(data))
data

<class 'dict'>


{'name': 'Leia Organa',
 'height': '150',
 'mass': '49',
 'hair_color': 'brown',
 'skin_color': 'light',
 'eye_color': 'brown',
 'birth_year': '19BBY',
 'gender': 'female',
 'homeworld': 'http://swapi.dev/api/planets/2/',
 'films': ['http://swapi.dev/api/films/1/',
  'http://swapi.dev/api/films/2/',
  'http://swapi.dev/api/films/3/',
  'http://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['http://swapi.dev/api/vehicles/30/'],
 'starships': [],
 'created': '2014-12-10T15:20:09.791000Z',
 'edited': '2014-12-20T21:17:50.315000Z',
 'url': 'http://swapi.dev/api/people/5/'}

Now we have a dictionary that we can work with.

Let's now take a look at another api. We'll start by looking at just the base URL:

In [10]:
base_url = 'https://python.zach.lol'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



This API provides some documentation, so let's make a request so that we can take a look at it.

In [27]:
# Create a new url
base_url + '/documentation'

'https://python.zach.lol/documentation'

In [13]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



Based on this, let's take a look at the items. We'll make our request, and explore the shape of the response that we get back.

In [14]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [16]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [19]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 3
next_page: /api/v1/items?page=2


Here the response has some built-in properties that tell us how to get to subsequent pages.

Once we've drilled down into the data structure, we'll find that the entire response is a sort of wrapper around the `items` property:

In [20]:
data['payload']['items'][:2]

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

We can turn this data into a pandas dataframe:

In [21]:
import pandas as pd

df = pd.DataFrame(data['payload']['items'])
df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [23]:
df.shape

(20, 6)

Now that we've gotten the data from the first page, we can extract the data from the next page (as indicated by the API's response), and add it onto our dataframe:

In [24]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()
df.shape

max_page: 3
next_page: /api/v1/items?page=3


(40, 7)

In [25]:
df.head()

,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


We'll repeat the process one more time:

In [26]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()
df.shape

max_page: 3
next_page: None


(50, 8)

Now that the API says that the `next_page` is None, we'll stop making requests, and assume that we have all of the `items` data.